In [3]:
import pandas as pd
from kinconnect_api.config import RAW_DATA_DIR, load_dotenv

load_dotenv()
fake_profile_data = pd.read_csv(f"{RAW_DATA_DIR}/fake_profile_data.csv")
fake_profile_data.head()

,"What are your interests? (ie technical topic, coding language, business problem).","If you have a project idea, describe your idea . Please include whether what sector it is in, and what business problem it is solving and for whom. If you don’t have a project, skip this question.","What is your strongest functional role (such as developer, UX, business, product)? Please share one or two things about your experience in role your experience, for example a success, how many years experience, a challenging project, etc.",Are you interested in meeting people with a specific skill set (either one that you lack or one that you already have but want to clone yourself to speed up building). What is the skills sets that you are looking to meet?
0,I'm interested in Rags and AI LLMs,NaN,"Back end developer, 10 years, build and deploy...","Product, frontend,"
1,I'm interested in using AI to solve climate ch...,How can AI use to regulate office temperature ...,Product manager. I worked as a product manager...,"Front end developer, Back end developer"
2,I'm interested in using AI to make building co...,Using AI to make building construction and dev...,Business Strategy. Business Development. Marke...,"Front end developer, Back end developer"
3,I'm interested in using AI to solve the loneli...,Match making,Frontend engineer. I worked at building their...,"Business strategy, Back end developer, product"
4,I'm interested in using AI to create more empathy,How can AI be used to make people think more a...,Backend engineer. I have worked on databases f...,Product managers. Strategic business. Maybe so...


In [18]:
profile_list = []

for index, row in fake_profile_data.iterrows():
    profile_dict = {'questions_and_answers': []}
    for column in fake_profile_data.columns:
        qa_pair = {"question": column, "answer": row[column]}
        profile_dict['questions_and_answers'].append(qa_pair)
    profile_list.append(profile_dict)

# Convert each profile's questions and answers into a formatted string and store in a list
formatted_profiles = [f'\n\n'.join(f"## {pair['question']}: \n {pair['answer']}" for pair in profile['questions_and_answers']) for profile in profile_list]

len(formatted_profiles)


12

In [20]:
print(formatted_profiles[1])

## What are your interests?  (ie technical topic, coding language, business problem).: 
 I'm interested in using AI to solve climate change

## If you have a project idea, describe your idea . Please include whether what sector it is in, and what business problem it is solving and for whom. If you don’t have a project, skip this question.: 
 How can AI use to regulate office temperature more efficiently depending on where people are in a building

## What is your strongest functional role (such as developer, UX, business, product)? Please share one or two things about your experience in role your experience, for example a success, how many years experience, a challenging project, etc.: 
 Product manager. I worked as a product manager at Uber, helping to build the mobile app for drivers. 

## Are you interested in meeting people with a specific skill set (either one that you lack or one that you already have but want to clone yourself to speed up building). What is the skills sets that 

In [29]:
from pymongo import MongoClient
from pydantic import BaseModel, Field
from typing import List, Dict

class CareerEntry(BaseModel):
    company: str = Field(..., title="Company they worked at")
    title: str = Field(..., title="Title of the role they held")
    description: str = Field(..., title="Description of the role they held")
    start_date: str = Field(..., title="Start date of the role")
    end_date: str = Field(..., title="End date of the role")


class ProjectEntry(BaseModel):
    title: str = Field(..., title="Title of the project")
    description: str = Field(..., title="Description of the project")


# Define a Pydantic model for the profile
class ProfileModel(BaseModel):
    name: str = Field(..., title="Name of the person")
    honors: str = Field(None, title="Honors, Awards and recognition they have recieved in life")
    interests: str = Field(..., title="Interests and current focus of theirs the work or the event")
    skills: str = Field(..., title="Skills they have")
    career: List[CareerEntry] = Field(..., title="Career history of the person")
    past_projects: List[ProjectEntry] = Field(..., title="Projects they have worked on")
    elevator_pitch: str = Field(..., title="Elevator pitch for the person fpr the event")
    
# # Connect to MongoDB
# client = MongoClient(MONGO_CONNECTION_STRING)
# db = client['profile_database']
# profiles_collection = db['profiles']

# # Create fake profiles
# fake_profiles = [
#     ProfileModel(
#         interests="AI and Machine Learning",
#         project_idea="Developing an AI to optimize energy usage in smart homes.",
#         functional_role="Data Scientist, 5 years experience in energy sector.",
#         skills_looking_for="IoT development skills"
#     ),
#     ProfileModel(
#         interests="Blockchain and Cryptocurrency",
#         project_idea="Creating a decentralized voting system.",
#         functional_role="Blockchain Developer, 3 years experience with Solidity.",
#         skills_looking_for="Frontend development skills"
#     ),
#     ProfileModel(
#         interests="Healthcare and Biotechnology",
#         project_idea="Using AI to predict disease outbreaks.",
#         functional_role="Bioinformatician, 7 years experience in epidemiology.",
#         skills_looking_for="Machine Learning and AI skills"
#     )
# ]

# # Insert fake profiles into MongoDB
# profiles_collection.insert_many([profile.dict() for profile in fake_profiles])


In [30]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chat_models.base import BaseChatModel
from textwrap import dedent
import logging
from pathlib import Path
import pandas as pd
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Dict, Callable, Union
import os
from tqdm import tqdm
from datetime import datetime
# Setup logging as per the new configuration

In [31]:
generate_fake_complete_prompt_str = dedent("""
You are a data generation app which creates profiles for participants at a hackathon. Given their form submission. Create a profile. If the information doesnt exist then feel free to create an interesting profile with relevant data.

                                       
{submission}

""")

In [28]:
from langchain_fireworks import Fireworks 
load_dotenv()

mixtral_llm = Fireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",)

structured_llm = mixtral_llm.with_structured_output(ProfileModel)

generate_fake_complete_prompt = ChatPromptTemplate.from_template(generate_fake_complete_prompt_str)

generate_fake_complete_chain = generate_fake_complete_prompt | structured_llm

generate_fake_complete_chain.invoke({"submission": formatted_profiles[0]})
# category_chat_prompt = ChatPromptTemplate.from_template(run_config['prompt'])


# categorize_chain = (category_chat_prompt | structured_llm).with_config({"tags": ["experiment"], "metadata": {"experiment_name": run_config['experiment_name']}})
# return categorize_chain

NameError: name 'ProfileModel' is not defined